In [1]:
from requests import get
import pandas as pd


In [2]:
all_countries=get('https://raw.githubusercontent.com/everypolitician/everypolitician-data/master/countries.json')
j=all_countries.json()
len(j)

233

In [3]:
aus_hr_url=j[13]['legislatures'][0]['popolo_url']
aus_hr_data=get(aus_hr_url)
aus_hr_df=pd.json_normalize(aus_hr_data.json()['persons'])

In [4]:
aus_hr_df['birth_date']=pd.to_datetime(aus_hr_df['birth_date'])
aus_hr_df.head()

,birth_date,contact_details,family_name,gender,given_name,id,identifiers,links,name,sort_name,death_date,other_names,email,image,images,honorific_prefix
0,1954-03-21,"[{'type': 'twitter', 'value': 'nueday'}]",Kelly,female,De-Anne,004e5075-34b7-4ba5-b53b-f992a28ea6dc,"[{'identifier': '10357', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",De-Anne Kelly,"Kelly, De-Anne",NaN,NaN,NaN,NaN,NaN,NaN
1,1925-06-15,NaN,Millar,male,Percival,00c2ed39-73db-43aa-a5b7-bea4508dbd5e,"[{'identifier': '10457', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Percival Millar,"Millar, Percival",2017-11-28,"[{'lang': 'en', 'name': 'Clarrie Millar', 'not...",NaN,NaN,NaN,NaN
2,1951-07-04,"[{'type': 'email', 'value': 'john.alexander.mp...",Alexander,male,John,010ba615-63c1-466b-a425-5d127e12b8c3,"[{'identifier': 'M3M', 'scheme': 'aph'}, {'ide...","[{'note': 'Wikimedia Commons', 'url': 'https:/...",John Alexander,"Alexander, John",NaN,"[{'lang': 'ar', 'name': 'جون ألكسندر', 'note':...",john.alexander.mp@aph.gov.au,http://www.aph.gov.au//api/parliamentarian/M3M...,[{'url': 'http://www.aph.gov.au//api/parliamen...,NaN
3,1954-02-13,"[{'type': 'twitter', 'value': 'senronno'}]",Ronaldson,male,Michael,012988dd-b93f-4bfe-95b8-27f3e5a56ee7,"[{'identifier': 'xt4', 'scheme': 'aph'}, {'ide...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Michael Ronaldson,"Ronaldson, Michael",NaN,"[{'lang': 'ja', 'name': 'マイケル・ロナルドソン', 'note':...",NaN,NaN,NaN,NaN
4,1946-08-28,NaN,Andren,male,Peter,0267b78a-440a-414f-bf35-5284b9ebaa0d,"[{'identifier': '10012', 'scheme': 'everypolit...","[{'note': 'Wikipedia (en)', 'url': 'https://en...",Peter Andren,"Andren, Peter",2007-11-03,NaN,NaN,NaN,NaN,NaN


In [5]:
aus_hr_df.shape

(515, 16)

In [6]:
aus_hr_df['birth_date']=pd.to_datetime(aus_hr_df['birth_date'])
monthly_df=aus_hr_df.groupby(aus_hr_df['birth_date'].dt.month).count()[['name']]
monthly_df.index=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_df['percentage']=(monthly_df['name']/monthly_df['name'].sum()*100).round(1).astype(str) + '%'
monthly_df.rename(columns={'name':'count'},inplace=True)
monthly_df.transpose()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,45,38,52,44,40,46,47,41,43,43,38,38
percentage,8.7%,7.4%,10.1%,8.5%,7.8%,8.9%,9.1%,8.0%,8.3%,8.3%,7.4%,7.4%


In [7]:
# Functions to get country and legislature wise data

def get_legislature_data(link):
    data=get(link)
    df=pd.json_normalize(data.json()['persons'])
    return df

def get_country_data(country):
    return pd.concat([get_legislature_data(legislature['popolo_url']) for legislature in country['legislatures']])

In [8]:
# Create a dataframe with data for all countries
df=pd.concat([get_country_data(country) for country in j])
df

KeyboardInterrupt: 

In [9]:
df.to_csv('data.csv',index=True)

NameError: name 'df' is not defined

In [15]:
df=pd.read_csv('data.csv')

C:\Users\Ojas\AppData\Local\Temp\ipykernel_30572\625188700.py:1: DtypeWarning: Columns (17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data.csv')


In [16]:
#Using string parsing to remove Jan skew
df['month']=df['birth_date'].astype(str).apply(lambda x: x[5:7])


In [17]:
monthly_df=df.groupby(df['month']).count()[['name']]
monthly_df.index=['Bad Data','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_df['percentage']=(monthly_df['name']/monthly_df['name'].sum()*100).round(1).astype(str) + '%'
monthly_df.rename(columns={'name':'count'},inplace=True)
monthly_df.transpose()

,Bad Data,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,34190,4876,3338,3742,3611,3724,3648,3657,3705,3602,3497,3337,3455
percentage,43.6%,6.2%,4.3%,4.8%,4.6%,4.8%,4.7%,4.7%,4.7%,4.6%,4.5%,4.3%,4.4%


In [18]:
#Previous implementation with January skew
#df['birth_date']=pd.to_datetime(df['birth_date'], errors='coerce') #y-m-d
#monthly_df=df.groupby(df['birth_date'].dt.month).count()[['name']]
#monthly_df.index=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
#monthly_df['percentage']=(monthly_df['name']/monthly_df['name'].sum()*100).round(1).astype(str) + '%'
#monthly_df.rename(columns={'name':'count'},inplace=True)
#monthly_df.transpose()

In [19]:
monthly_df.drop('Bad Data',inplace=True)
monthly_df['percentage']=(monthly_df['count']/monthly_df['count'].sum()*100).round(2).astype(str) + '%'
monthly_df

,count,percentage
Jan,4876,11.0%
Feb,3338,7.6%
Mar,3742,8.5%
Apr,3611,8.2%
May,3724,8.4%
Jun,3648,8.3%
Jul,3657,8.3%
Aug,3705,8.4%
Sep,3602,8.2%
Oct,3497,7.9%


In [20]:
monthly_df['expected']=[31/365,28/365,31/365,30/365,31/365,30/365,31/365,31/365,30/365,31/365,30/365,31/365]
monthly_df['expected']=(monthly_df['expected']/monthly_df['expected'].sum()*100).round(2).astype(str) + '%'
monthly_df

,count,percentage,expected
Jan,4876,11.0%,8.5%
Feb,3338,7.6%,7.7%
Mar,3742,8.5%,8.5%
Apr,3611,8.2%,8.2%
May,3724,8.4%,8.5%
Jun,3648,8.3%,8.2%
Jul,3657,8.3%,8.5%
Aug,3705,8.4%,8.5%
Sep,3602,8.2%,8.2%
Oct,3497,7.9%,8.5%


In [21]:
births_data=pd.read_csv('Births Data UN.csv')
births_data

,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2021.0,Total,Total,Data tabulated by year of occurrence,Provisional figure,2022.0,293.0,NaN
1,Åland Islands,2021.0,Total,January - March,Data tabulated by year of occurrence,Provisional figure,2021.0,70.0,NaN
2,Åland Islands,2021.0,Total,April - June,Data tabulated by year of occurrence,Provisional figure,2021.0,73.0,NaN
3,Åland Islands,2021.0,Total,July - September,Data tabulated by year of occurrence,Provisional figure,2021.0,82.0,NaN
4,Åland Islands,2021.0,Total,October - December,Data tabulated by year of occurrence,Provisional figure,2022.0,68.0,NaN
...,...,...,...,...,...,...,...,...,...
54844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
births_data.drop(['Area', 'Record Type', 'Value Footnotes', 'Source Year'],axis=1,inplace=True)
births_data

,Country or Area,Year,Month,Reliability,Value
0,Åland Islands,2021.0,Total,Provisional figure,293.0
1,Åland Islands,2021.0,January - March,Provisional figure,70.0
2,Åland Islands,2021.0,April - June,Provisional figure,73.0
3,Åland Islands,2021.0,July - September,Provisional figure,82.0
4,Åland Islands,2021.0,October - December,Provisional figure,68.0
...,...,...,...,...,...
54844,NaN,NaN,NaN,NaN,NaN
54845,NaN,NaN,NaN,NaN,NaN
54846,NaN,NaN,NaN,NaN,NaN
54847,NaN,NaN,NaN,NaN,NaN


In [23]:
births_data=births_data.query("Month in ['January','February','March','April','May','June','July','August','September','October','November','December']")
births_data

,Country or Area,Year,Month,Reliability,Value
6,Åland Islands,2020.0,January,"Final figure, complete",11.0
7,Åland Islands,2020.0,February,"Final figure, complete",24.0
8,Åland Islands,2020.0,March,"Final figure, complete",31.0
9,Åland Islands,2020.0,April,"Final figure, complete",22.0
10,Åland Islands,2020.0,May,"Final figure, complete",20.0
...,...,...,...,...,...
54796,Wallis and Futuna Islands,1973.0,August,"Final figure, complete",364.0
54797,Wallis and Futuna Islands,1973.0,September,"Final figure, complete",300.0
54798,Wallis and Futuna Islands,1973.0,October,"Final figure, complete",385.0
54799,Wallis and Futuna Islands,1973.0,November,"Final figure, complete",344.0


In [35]:
births_df=births_data.groupby('Month').sum()[['Value']]
births_df

,Value
Month,
April,8.449080e+07
August,9.257204e+07
December,8.634094e+07
February,8.228612e+07
January,9.274157e+07
July,9.187408e+07
June,8.627537e+07
March,8.805267e+07
May,8.731629e+07


In [36]:
births_df=births_df.reindex(['January','February','March','April','May','June','July','August','September','October','November','December'])

In [37]:
births_df.index=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
births_df

,Value
Jan,9.274157e+07
Feb,8.228612e+07
Mar,8.805267e+07
Apr,8.449080e+07
May,8.731629e+07
Jun,8.627537e+07
Jul,9.187408e+07
Aug,9.257204e+07
Sep,9.286552e+07
Oct,9.115465e+07


In [38]:
births_df['births_percentage']=(births_df['Value']/births_df['Value'].sum()*100).round(2).astype(str) + '%'
births_df.rename(columns={'Value':'births_count'},inplace=True)
births_df.drop('births_count',axis=1,inplace=True)

In [39]:
df=pd.merge(monthly_df,births_df,how='inner',left_index=True,right_index=True)
df

,count,percentage,expected,births_percentage
Jan,4876,11.0%,8.5%,8.73%
Feb,3338,7.6%,7.7%,7.75%
Mar,3742,8.5%,8.5%,8.29%
Apr,3611,8.2%,8.2%,7.95%
May,3724,8.4%,8.5%,8.22%
Jun,3648,8.3%,8.2%,8.12%
Jul,3657,8.3%,8.5%,8.65%
Aug,3705,8.4%,8.5%,8.72%
Sep,3602,8.2%,8.2%,8.74%
Oct,3497,7.9%,8.5%,8.58%


In [40]:
df['percentage']=df['percentage'].str.replace('%','').astype(float)
df['births_percentage']=df['births_percentage'].str.replace('%','').astype(float)
df['expected']=df['expected'].str.replace('%','').astype(float)
df['difference']=df['percentage']-df['births_percentage']
df['pc_difference']=df['difference']/df['births_percentage']*100
df['pc_difference']=df['pc_difference'].round(2)
df

,count,percentage,expected,births_percentage,difference,pc_difference
Jan,4876,11.0,8.5,8.73,2.27,26.00
Feb,3338,7.6,7.7,7.75,-0.15,-1.94
Mar,3742,8.5,8.5,8.29,0.21,2.53
Apr,3611,8.2,8.2,7.95,0.25,3.14
May,3724,8.4,8.5,8.22,0.18,2.19
Jun,3648,8.3,8.2,8.12,0.18,2.22
Jul,3657,8.3,8.5,8.65,-0.35,-4.05
Aug,3705,8.4,8.5,8.72,-0.32,-3.67
Sep,3602,8.2,8.2,8.74,-0.54,-6.18
Oct,3497,7.9,8.5,8.58,-0.68,-7.93


In [42]:
from plotnine import *